In [1]:
import numpy as np
import string
import time
import os

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

Using Theano backend.


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

In [10]:
os.listdir('/root/facial_expressions/data')

['500_picts_satz.csv', 'legend.csv']

### Load Training and Test Data

In [3]:
emoMetaData = dwdii_transforms.load_training_metadata(dataPath)

In [4]:
len(emoMetaData)

13682

In [13]:
maxData = 2000
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 200)
print X_data.shape
print Y_data.shape

0.000000: King_Abdullah_II_0001.jpg
0.010000: Giuseppe_Gibilisco_0003.jpg
0.020000: Don_Lake_0001.jpg
0.030000: Jason_Priestley_0001.jpg
0.040000: Shaukat_Aziz_0002.jpg
0.050000: Jim_Hardin_0001.jpg
0.060000: Ana_Palacio_0003.jpg
0.070000: Nicolas_Escude_0002.jpg
0.080000: Janet_Crawford_0001.jpg
0.090000: Jayamadhuri_59.jpg
0.090000: Taufik_Hidayat_0001.jpg
0.100000: George_W_Bush_0193.jpg
0.110000: Andres_Pastrana_0001.jpg
0.120000: Owen_Wilson_0002.jpg
0.130000: Jackie_Chan_0004.jpg
0.140000: Laura_Bush_0003.jpg
0.150000: Kate_Hudson_0009.jpg
0.160000: Anders_Ebbeson_0001.jpg
0.170000: Milton_Berle_0001.jpg
0.180000: Rudolph_Giuliani_0005.jpg
0.190000: Eugene_Teslovic_0001.jpg
0.200000: Jason_Alexander_0002.jpg
0.210000: Amber_Tamblyn_0001.jpg
0.220000: Yann_Martel_0001.jpg
0.230000: Jeannette_Biedermann_0001.jpg
0.240000: Heather_Locklear_0001.jpg
0.250000: Naji_Sabri_0003.jpg
0.260000: George_W_Bush_0059.jpg
0.270000: Paradorn_Srichaphan_0005.jpg
0.280000: Paul_Wolfowitz_0004.jpg


The following code segment splits the data into training and test data sets. Currently this is just a standard 80/20 split for training and test respectively. A random.shuffle should be probably be added here as well.

In [14]:
# Split the data into Training and Test sets
trainNdx = int(maxData * .8)
print trainNdx
X_train, X_test = np.split(X_data, [trainNdx])
Y_train, Y_test = np.split(Y_data, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

1600
(1600, 350, 350)
(400, 350, 350)
(1600, 1)
(400, 1)


### Transformations
In this section, we will apply transformations to the existing images to increase of training data, as well as add a bit of noise in the hopes of improving the overall training activities.

### Define the Model
In this section, we define the model. The `emotion_model` module contains the model definition itself. `emotion_model_v1` is a basic convolutional neural network.

In [15]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
7


In [9]:
model = emotion_model.emotion_model_v1(len(emotions), verbose=True)

Compiling Model ... 
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)    (None, 1, 352, 352) 0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)    (None, 32, 350, 350)320         zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 175, 175)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)    (None, 32, 177, 177)0           maxpooling2d_1[0][0]             
______________________________________________________________________

### Training the Model

The following code segment trains the model using the `run_network` helper function. I seem to be hitting a memory issue (my interpretation), when I have batches above a certain threshold. Batches=10 work fine, but batches of 100 are too big. May need to allocate more RAM to the docker container.

In [ ]:
testX = X_test.reshape(X_test.shape[0], 1, 350, 350)
trainX = X_train.reshape(X_train.shape[0], 1, 350, 350)

emotion_model.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=10)

(1600, 7)
(400, 7)
Training model...
Train on 1600 samples, validate on 400 samples
Epoch 1/10


In [ ]:
model.save_weights("dwdii-emo-01v1.hdf5", overwrite=True)

### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits
  * docker ps -l
  * docker commit <ContainerID> dittenhafer/dl